In [17]:
import firebase_admin
from firebase_admin import credentials, db, storage
import csv
from datetime import datetime
import numpy as np
import cv2

In [18]:
# Fetch the service account key JSON file contents
cred = credentials.Certificate('rhizobase_key2.json')

# Initialize the app with a service account, granting admin privileges
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://rhizobase-7595e-default-rtdb.asia-southeast1.firebasedatabase.app/',
        'storageBucket': 'rhizobase-7595e.appspot.com'
    })

# As an admin, the app has access to read and write all data, regradless of Security Rules
ref = db.reference('')

In [19]:
#Get Existing Keys
def key_get():    
    ref = db.reference('keys')
    info = ref.get()
    return info

info = key_get()

In [58]:
# Messages Compilation
menu = {
    "create_user": "Account Creation",
    "login": "\nWelcome to RhizoBase.\nPlease enter 1 to log in or 2 to sign up",
    "start": "\nWelcome to RhizoBase.\n1 - View Rhizo Albums\n2 - Submit Rhizo Album\n3 - Submit Other Data\n4 - Exit",
    "view1": "1 - View Uploaded Images\n2 - View Preprocessed Images",
    "exit": "Thank you for using RhizoBase.",
    "menu-2": "Submit Rhizotron Image Album",
    "menu-3": "Submit Supporting Information\n1 - Location Data\n2 - Imaging Device Data\n3 - Rhizo Set Up Data",
    "new dataset": "Submit a new rhizotron dataset"
}
request = {
    "login":"Choice: ",
    "username": "Please enter account name: ",
    "password": "Please enter password: ",
    "start": "Enter choice: ",
    "view1": "Enter choice: ",
    "menu-3": "Enter choice: ",
    "ds name": "Enter dataset name: ",
    "ds description": "Enter dataset description: ",
    "ds study": "Enter associated study with this dataset, if any: "
}
choices = {
    "login":["1","2"],
    "start": ["1","2","3","4"],
    "menu-3": ["1","2","3"],
    "view1": ["1","2"]
}

invalid = "Invalid Input!"



In [59]:
def message(idx):
    if idx in menu:
        print(menu[idx] + "\n")
    if idx in request:
        while True:
            try:
                answer = input(request[idx])
            except ValueError:
                print(invalid)
                continue
            if idx in choices:
                if answer in choices[idx]:
                    return answer
                    break
                else:
                    print(invalid)
            else:
                return answer
            
def convert_message(word_list, topic):
    string = "Available Choices for " + topic + ":\n"
    prompt = "Enter choice: "
    choices = []
    for idx in range(len(word_list)):
        choices.append(str(idx+1))
        string = string + str(idx+1) + " - " + word_list[idx] + "\n" 
    return string, prompt, choices

In [22]:
loc = "dataset album"
dataset_list = gather_names(loc,info["dataset_id"],"dataset_name")
menu[loc],request[loc],choices[loc] = convert_message(dataset_list, loc)
selected_dataset_id = message(loc)

Available Choices for dataset album:
1 - Sample Dataset


Enter choice: 1


'1'

In [75]:
def input_values(loc, idnum):
    ref = db.reference(loc)
    dic = ref.child('0').get()
    new_data = {}
    for key in dic.keys():
        new_data[key] = input(key+": ")
    ref.update({
        idnum:new_data
    })

def process_image(image, location, width, height, bw, threshold):
    img = cv2.imread(location+image)
    if bw == "y":
        thresh, img = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(location+"preprocessed_"+image, img)    
    
def rhizo_csv(loc, file, dataset_id):    
    with open(loc+file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        header = True
        process_width = int(input("Enter processed image width: "))
        process_height = int(input("Enter processed image height: "))
        process_bw = input("Convert to black and white? [y/n]: ")
        process_threshold = "n/a"
        if process_bw == "y":
            process_threshold = int(input("Enter white threshold: "))
        for row in csv_reader:
            if header == True:
                header = False
            else:
                info = key_get()
                idnum = info["rhizo_image_id"]
                ref = db.reference("rhizo_image")
                new_data = {}
                new_data["image_name"] = row[0]
                new_data["age"] = row[1]
                new_data["is_greyscale?"] = row[2]
                new_data["format"] = row[3]
                new_data["rhizo_setup_id"] = row[4]
                new_data["strain_id"] = row[5]
                new_data["dataset_id"] = dataset_id
                ref.update({
                    idnum:new_data
                })
                
                pre_idnum = info["preprocess_id"]
                ref = db.reference("preprocessed_image")
                new_data = {}
                new_data["resize_dimension"] = str("("+str(process_width)+","+str(process_height)+")")
                new_data["black and white"] = process_bw
                new_data["threshold_method"] = process_threshold
                new_data["preprocess_rhizo_name"] = "preprocessed_"+row[0]
                new_data["rhizo_image_id"] = idnum
                ref.update({
                    pre_idnum:new_data
                })
                
                fileName = row[0]
                location = "rhizobase/"
                bucket = storage.bucket()
                blob = bucket.blob("Uploaded Image/"+fileName)
                blob.upload_from_filename(location+fileName)
                blob.make_public()
                print("Uploaded image "+ row[0])
                key_update("rhizo_image_id",idnum+1)
                
                process_image(fileName, location, process_width, process_height, process_bw, process_threshold)
                bucket = storage.bucket()
                blob = bucket.blob("Processed Image/"+"preprocessed_"+fileName)
                blob.upload_from_filename(location+"preprocessed_"+fileName)
                blob.make_public()
                print("Processed image " + row[0])
                key_update("preprocess_id",idnum+1)
                
                

In [74]:
rhizo_csv("RhizoBase/","rhizo_album1.csv",info["dataset_id"])
#print(datetime.now().strftime("%m/%d/%Y"))

Enter processed image width: 300
Enter processed image height: 500
Convert to black and white? [y/n]: y
Enter white threshold: 48
Uploaded image root_1.png
Processed image root_1.png
Uploaded image root_2.png
Processed image root_2.png
Uploaded image root_3.png
Processed image root_3.png


In [61]:
def key_update(code,new):
    ref = db.reference("keys")
    ref.update({
        code:new
    })

def gather_names(loc, idnum, value):
    list = []
    for idx in range(idnum):
            idx = int(idx)
            if idx > 0:
                ref = db.reference(loc)
                data_dict = ref.child(str(idx)).get()
                data = data_dict[value]
                list.append(data)
    return list

def ab_dict(loc, idnum, value_a, value_b):
    dic = {}
    for idx in range(idnum):
        idx = int(idx)
        if idx > 0:
            ref = db.reference(loc)
            data = ref.child(str(idx)).get()
            a = data[value_a]
            b = data[value_b]
            dic[a] = b
    return dic, a

In [62]:
def find_img(dataset_id):
    img_dict = {}
    ref = db.reference("rhizo_image")
    all_img = ref.get()
    for idx in range(len(all_img)):
            idx = int(idx)
            if idx > 0:
                img_data_id = all_img[idx]["dataset_id"]
                if dataset_id == img_data_id:
                     img_dict[idx] = all_img[idx]["image_name"]
    return img_dict

def find_prepro(dataset_id):
    img_dict = {}
    ref = db.reference("rhizo_image")
    all_img = ref.get()
    ref = db.reference("preprocessed_image")
    all_pre = ref.get()
    for idx in range(len(all_img)):
        idx = int(idx)
        if idx > 0:
            img_data_id = all_img[idx]["dataset_id"]
            if dataset_id == img_data_id:
                for idy in range(len(all_pre)):
                    idy = int(idy)
                    if idy > 0:
                        orig_img_id = all_pre[idx]["rhizo_image_id"]
                        if orig_img_id == idx:
                            img_dict[idy] = all_pre[idx]["preprocess_rhizo_name"]
    return img_dict

def display_img(dic, loc):
    img_ids = dic.keys()
    for idx in img_ids:
        print(str(idx)+" - "+dic[idx])
    while True:
        try:
            answer = input("Select image to view [input number]: ")
        except ValueError:
            print(invalid)
            continue
        if int(answer) in img_ids:
            bucket = storage.bucket()
            blob = bucket.get_blob(loc+dic[int(answer)])
            arr = np.frombuffer(blob.download_as_string(),np.uint8)
            img = cv2.imdecode(arr, cv2.COLOR_BGR2BGR555)
            cv2.imshow("image",img)
            cv2.waitKey()
            cv2.startWindowThread()
            break
        else:
            print(invalid)
           

In [72]:
print(find_img(1))
print(find_prepro(1))
display_img(find_prepro(1),"Processed Image/")

{1: 'root_1.png', 2: 'root_2.png', 3: 'root_3.png'}
{1: 'preprocessed_root_3.png', 2: 'preprocessed_root_3.png', 3: 'preprocessed_root_3.png'}
1 - preprocessed_root_3.png
2 - preprocessed_root_3.png
3 - preprocessed_root_3.png
Select image to view [input number]: 3


In [69]:
#Log In
login = False
while login == False:
    loc = "login"
    choice = message(loc)

    if choice == "1":
        ref = db.reference("keys")
        idnum = ref.child("user_id").get()
        login_detail, login_list = ab_dict("user", idnum, "account name", "password")
        login_acc = message("username")
        login_pw = message("password")
        if login_detail[login_acc] == login_pw:
            login = True
            current_user_id = 1+int(login_list.index(login_acc))
            print("Log in success!")
        else:
            print(invalid)
    elif choice == "2":
        key = info["user_id"]
        message("create_user")
        input_values("user", key)
        key_update("user_id",key+1)
        print("Returning to Log In...")
    


Welcome to RhizoBase.
Please enter 1 to log in or 2 to sign up

Choice: 1
Please enter account name: jiego
Please enter password: pw
Log in success!


In [76]:
# "Interface Proper"
while login == True:
    loc = "start"
    ans = message(loc)
    if ans == '1':
        loc = "dataset album"
        dataset_list = gather_names(loc,info["dataset_id"],"dataset_name")
        menu[loc],request[loc],choices[loc] = convert_message(dataset_list, loc)
        selected_dataset_id = message(loc)
        
        loc = "view1"
        img_type = message(loc)
        if img_type == "1":
            img_list = (find_img(1))
            file_loc = "Uploaded Image/"
        else:
            file_loc = "Processed Image/"
            img_list = (find_prepro(1))
        display_img(img_list,file_loc)
    elif ans == '2':
        loc = "new dataset"
        message(loc)
        
        loc = "ds name"
        dataset_name = message(loc)
        loc = "ds description"
        dataset_desc = message(loc)
        
        info = key_get()
        dataset_id = info["dataset_id"]
        loc = "rhizotron setup"
        rhizo_setup_list = gather_names(loc,info["rhizo_setup_id"],"setup name")
        menu[loc],request[loc],choices[loc] = convert_message(rhizo_setup_list, "rhizotron setup")
        selected_rhizo_setup_id = message(loc) 
        
        loc = "location"
        rhizo_setup_list = gather_names(loc,info["location_id"],"location description")
        menu[loc],request[loc],choices[loc] = convert_message(rhizo_setup_list, "study location")
        selected_location_id = message(loc) 
        
        today = datetime.now().strftime("%m/%d/%Y")
        csv_dir = input("Enter csv directory for image data: ")
        csv_name = input("Enter csv name for image data: ")
        
        loc = "ds study"
        dataset_study = message(loc)
        #convert_message
        rhizo_csv(csv_dir, csv_name, dataset_id)
        
        ref = db.reference("dataset album")
        ref.update({
            dataset_id:{
                "rhizo_setup_id":selected_rhizo_setup_id,
                "user_id": current_user_id,
                "location_id": selected_location_id,
                "dataset_name": dataset_name,
                "upload_date": today,
                "dataset_description": dataset_desc,
                "associated_study": dataset_study               
            }
        })
        key_update("dataset_id",dataset_id+1)
        
        print("Dataset uploaded.")
    elif ans == '3':
        loc = "menu-3"
        ans3 = message(loc)
        if ans3 == '1':
            idtype = "location_id"
            val = "location"
            print("Input Location Information")
        elif ans3 == '2':
            idtype = "device_id"
            val = "device"
            print("Input Imaging Device Information")
        elif ans3 == '3':
            idtype = "rhizo_setup_id"
            val = "rhizotron setup"
            print("Input Rhizotron Setup Information")
        key = info[idtype]
        input_values(val, key)
        key_update(idtype,key+1)
        info = key_get()
    else:
        message("exit")
        login = False


Welcome to RhizoBase.
1 - View Rhizo Albums
2 - Submit Rhizo Album
3 - Submit Other Data
4 - Exit

Enter choice: 1
Available Choices for dataset album:
1 - Sample Dataset


Enter choice: 1
1 - View Uploaded Images
2 - View Preprocessed Images

Enter choice: 2
1 - preprocessed_root_3.png
2 - preprocessed_root_3.png
3 - preprocessed_root_3.png
Select image to view [input number]: 1

Welcome to RhizoBase.
1 - View Rhizo Albums
2 - Submit Rhizo Album
3 - Submit Other Data
4 - Exit

Enter choice: 4
Thank you for using RhizoBase.



In [ ]:
# "Interface Proper"
if login == True:
    loc = "start"
    ans = message(loc)
    if ans == '1':
        print("view")
    elif ans == '2':
        print("new")
    elif ans == '3':
        print("a")
    else:
        message("exit")
        login = False

In [14]:
print(info)

{'dataset_id': 1, 'device_id': 3, 'location_id': 3, 'pred_trl_id': 1, 'preprocess_id': 1, 'rhizo_image_id': 1, 'rhizo_setup_id': 3, 'species_id': 3, 'strain_id': 2, 'trl_model_id': 1, 'user_id': 2}


print(login_detail)